In [1]:
from data import Datagenerator, TrainingStats
from model import LEO, load_model, save_model
from  torch.nn import MSELoss
from easydict import EasyDict as edict
from utils import load_config, check_experiment
from run import train_model
import torch.optim as optim
import argparse
import torch 
import torch.optim
import gc
from utils import display_data_shape, get_named_dict, one_hot_target,\
    softmax, sparse_crossentropy, calc_iou_per_class, log_data, load_config
import matplotlib.pyplot as plt
import numpy as np

c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Train

In [2]:
config = load_config()

In [ ]:
leo, meta_data, classes = train_model(config)

num tasks: 7
tr_data shape: torch.Size([7, 2, 384, 512, 14]),tr_data_masks shape: torch.Size([7, 2, 384, 512]), val_data shape: torch.Size([7, 5, 384, 512, 14]),val_masks shape: torch.Size([7, 5, 384, 512])


C:\Users\temil\OneDrive\Documents\Study\UHildesheim\Project\Scripts\leo-srp-project\merge_to_dev\leo_segmentation\leo_segmentation\model.py:92: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output channels_last tensor even if some of the inputs are not in channels_last format. (Triggered internally at  ..\aten\src\ATen\native\TensorIterator.cpp:924.)
  channel_zero = inputs * predicted_weights[:, :14, :, :]
c:\users\temil\onedrive\documents\codes_and_scripts\python_envs\pytorch-tf2-env\lib\site-packages\torch\autograd\__init__.py:192: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  ..\aten\src\ATen\native\TensorIterator.cpp:918.)
  inputs, allow_unused)


Class: bus, Episode: 1, Val IOU: 0.17216083276237387
Class: cow, Episode: 1, Val IOU: 0.13238326732092848
Class: bottle, Episode: 1, Val IOU: 0.15815690579669434
Class: boat, Episode: 1, Val IOU: 0.056805459646771285
Class: aeroplane, Episode: 1, Val IOU: 0.07448769727063236
Class: cat, Episode: 1, Val IOU: 0.19199552557704777
Class: car, Episode: 1, Val IOU: 0.10829365725986032

mode:meta_train, episode:001, kl_loss:0.128264, total_val_loss:0.774303


## Visualize Predictions

In [ ]:
leo

In [ ]:
classes

In [ ]:
select_class = 2
print("class", classes[select_class], "is selected")
select_sample = 1
batch_data = get_named_dict(meta_data, select_class)
input_data = batch_data.val_data[select_sample].permute(1, 2, 0)
plt.title("First three channels of embeddings is visualized")
plt.imshow(input_data.cpu().detach().numpy()[:,:,:3])

In [ ]:
input_mask = batch_data.val_data_masks[select_sample]
plt.title("ground truth mask")
plt.imshow(input_mask.cpu().detach().numpy(), cmap="gray")

In [ ]:
latents, kl_loss = leo.forward_encoder(batch_data.val_data)
_, _, predictions = leo.forward_decoder(batch_data.val_data, latents, batch_data.val_data_masks)

In [ ]:
plt.title("mask prediction")
plt.imshow(np.argmax(predictions.cpu().detach().numpy()[select_sample], 0), cmap="gray")

In [ ]:
predictions.shape